In [62]:
import numpy as np
import pandas as pd

# Movie Revenue Predictions


Onderzoeksvragen:
1. In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?
2. In hoeverre is het mogelijk om op basis van schrijvers en regisseur te voorspellen welke acteurs in een film spelen?
3. In hoeverre is het mogelijk om op basis van plot keywords te voorspellen welke genres de film behoort?
4. In hoeverre is het mogelijk om de budget-winst verhouding te voorspellen?

### Concept vragen, om te verwijderen.

Drie concept-onderzoeksvragen:
1. In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf? (min. eisen 4.?)
2. In hoeverre is het mogelijk om op basis van plot keywords te voorspellen welke genres de film heeft? (min. eisen 5.?)
3. In hoeverre is de verhouding budget-omzet veranderd sinds 1935? (min. eisen 4.)

Externe data (minimale eisen 1.)
4. In hoeverre is het mogelijk om op basis van schrijvers en regisseur te voorspellen welke acteurs in een film spelen? (min. eisen 6.?)
5. In hoeverre kunnen wij voorspellen wanneer het budget en omzet hun zenit bereiken gebaseerd op de wereldpopulatie. (min. eisen 5.?)

## Het Data Science proces
Voor de eerste verkenning is ons gevraagd om de eerste vier stappen uit te voeren:
1. Data collection
2. Data processing (ook wel data munging)
3. Data cleaning
4. Data exploration & analysis
5. Model building
6. Visualization
6. Communication


## 1. Data Collection
De Data Collection is deels al voor ons gedaan. De dataset `movie.csv` is ons aangeleverd. Echter word voor de opdracht 
gevraagd om deze te combineren met een dataset van derden. Om erachter te komen welke dataset geschikt is om te 
combineren met `movies.csv`zullen wij deze dataset eerst moeten processen, cleanen en exploren.  

Om te zien of de dataset `movie.csv` goed is ingeladen, worden de eerste vijf rijen getoond:

In [63]:
df_movies = pd.read_csv('data/movie.csv')
df_movies.head()

### Externe dataset
Onze originele dataset bevat geen data over de schrijvers van deze films. Deze data valt echter wel te reconstrueren met
behulp van andere IMDB datasets. Het gaat hierbij om de volgende datasets:

In [3]:
# These datasets were originally named 'data.tsv', they have been renamed accordingly:
# title.crew.tsv.gz -> crew.tsv
df_crew = pd.read_csv(r"data\imdb\crew.tsv", sep="\t")
df_crew.head()

In [4]:
# name.basics.tsv.gz -> names.tsv
df_names = pd.read_csv(r"data/imdb/names.tsv", sep="\t")
df_names.head()


In [5]:
# title.basics.tst.gz -> titles.tsv
df_titles = pd.read_csv(r"data/imdb/titles.tsv", sep="\t")
df_titles.head()

## 2. Data Processing
### 2a. Aangeleverde dataset
Ook deze stap is grotendeels voor ons gedaan. De data is goed opgeslagen in een `.csv`-bestand en kan direct worden 
ingelezen in een _Pandas_ DataFrame.

Verder rest ons nog de volgende vier stappen:
1. Het verwijderen van ongewenste kolommen
2. Het aanpassen van onduidelijke kolomnamen
3. Het aanpassen van de volgorde van de kolommen
4. Het aanpassen van enkele datatypen

In [64]:
# Ongewenste kolommen verwijderen
df_movies.drop(["movie_imdb_link", "aspect_ratio"], axis=1, inplace=True)

#Onduidelijke kolomnamen aanpassen
df_movies.rename(columns={'color': 'Colour',
                          'director_name': 'Director',
                          'num_critic_for_reviews': 'Number of critics',
                          'duration': 'Duration',
                          'director_facebook_likes': 'Director FB likes',
                          'actor_3_facebook_likes': 'Actor 3 FB likes',
                          'actor_2_name': 'Actor 2 name',
                          'actor_1_facebook_likes': 'Actor 1 FB likes',
                          'gross': 'Gross',
                          'genres': 'Genres',
                          'actor_1_name': 'Actor 1 name',
                          'movie_title': 'Movie title',
                          'num_voted_users': 'Number of voted users',
                          'cast_total_facebook_likes': 'Total Cast FB likes',
                          'actor_3_name': 'Actor 3 name',
                          'facenumber_in_poster': 'Number of faces on poster',
                          'plot_keywords': 'Plot Keywords',
                          'num_user_for_reviews': 'Number of user reviews',
                          'language': 'Language',
                          'country': 'Country',
                          'content_rating': 'Age rating',
                          'budget': 'Budget',
                          'title_year': 'Release year',
                          'actor_2_facebook_likes': 'Actor 2 FB likes',
                          'imdb_score': 'IMDB Score',
                          'movie_facebook_likes': 'Movie FB likes'}, inplace=True)

# Volgorde kolommen aanpassen
df_movies = df_movies[['Movie title',
                       'Release year',
                       'Director',
                       'Director FB likes',
                       'Movie FB likes',
                       'Gross',
                       'Budget',
                       'Duration',
                       'Language',
                       'Country',
                       'Colour',
                       'Genres',
                       'IMDB Score',
                       'Number of voted users',
                       'Number of critics',
                       'Number of user reviews',
                       'Age rating',
                       'Total Cast FB likes',
                       'Actor 1 name',
                       'Actor 2 name',
                       'Actor 3 name',
                       'Actor 1 FB likes',
                       'Actor 2 FB likes',
                       'Actor 3 FB likes',
                       'Plot Keywords',
                       'Number of faces on poster',
                       ]]

# Datatypen aanpassen
# 1. Floats omzetten naar integers
#  De dataset bevat geen kolommen die dienen te worden bewaard als float, behalve `IMDB Score`
df_movies_IMDB_Score = df_movies["IMDB Score"]  # Tijdelijke kopie van de kolom `IMDB Score`
df_movies = df_movies.drop('IMDB Score', axis=1).fillna(0.0).astype(int, errors='ignore') # Waarden omzetten naar integers
df_movies.insert(12, "IMDB Score", df_movies_IMDB_Score)  # `IMDB Score` weer toevoegen aan originele DataFrame
del df_movies_IMDB_Score

# 2. De kolom `Release year` omzettten van integers naar het datetime-datatype
df_movies["Release year"] = pd.to_datetime(df_movies["Release year"], format='%Y', errors='coerce')

Het DataFrame ziet er nu als volgt uit:

In [65]:
# print(df_movies["IMDB Score"][:5])  # Debugging
df_movies.head()

### 2b. Extra datasets
Uit de extra datasets moeten een DataFrame worden gemaakt die de volgende kolommen bevat:  
- Movie title
- Director
- Writers
- Actor 1 name
- Actor 2 name
- Actor 3 name

Dit gaat in een paar stappen:
1. Het verwijderen van kolommen die niet gebruikt worden
2. Het samenvoegen van `df_crew` en `df_titles` op `tconst`
3. Het verwijderen van alle rijen die geen films zijn
4. Het toevoegen van de namen van de schrijvers uit `df_names`  
5. Het toevoegen van de kolommen 'Actor # name' uit `df_movies`
6. Het toevoegen van de naam van de directors



In [8]:
# Removing unused columns
df_titles.drop(["isAdult", "endYear", "genres"], axis=1, inplace=True)

In [9]:
# Joining df_crew and df_titles
df_writers = pd.DataFrame(pd.merge(df_crew, df_titles))

In [10]:
# Dropping all non-movies
df_writers.drop(df_writers[df_writers["titleType"] != "movie"].index, inplace=True)

In [11]:
# Adding names of writers to `df_writers`


In [12]:
# Adding "Actor # name" 1 through 3 to `df_writers`


Here's a sample of the current version of `df_writers` 

In [13]:
df_writers.sample(5)

## 3. Data Cleaning
### 3a. Aangeleverde dataset
1. NaN-types verwijderen
2. Dubbele Movie Titles verwijderen

In [14]:
# NaN-types verwijderen
df_movies.dropna(inplace=True)

# Dubbele titels verwijderen

Na stap 3. Data Cleaning ziet het DataFrame er als volgt uit:

In [15]:
df_movies.head()

### 3a. Extra datasets
1. "\\N" waardes omzetten naar NaN-types
2. NaN-types verwijderen
3. Dubbele Movie Titles verwijderen


In [16]:
# Replacing "\\N" with None
df_writers.replace("\\N", np.NaN, inplace=True)
# Dropping all rows with NaN-types
df_writers.dropna(inplace=True)

Na stap 3. Data Cleaning ziet `df_writers` er zo uit:

In [17]:
df_writers.sample(5)

## 4. Data Exploration & Analysis

In [18]:
df_movies["Director"].value_counts()

In [19]:
df_movies["Actor 1 name"].value_counts()

In [20]:
df_movies["Genres"].sample(10)

In [21]:
# W.I.P. Mogelijk in de toekomst nog een toepassing voor:
# df_movies.describe(include = "all")
# df_movies.set_index("Movie title", inplace=True)
# df_movies.sort_index(inplace=True)